# BioExplorer - Synapses
![](../bioexplorer_neurons_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

In [ ]:
guid = 300
population_name = 'o1'

### Neuron

In [ ]:
assembly_name = '%d' % guid
sql_filter = 'guid=%d' % (guid - 0)
be.remove_assembly(assembly_name)
neurons_assembly = be.add_assembly(assembly_name)
neurons_model = be.add_neurons(
    assembly_name=assembly_name,
    population_name=population_name,
    load_somas=True, load_axon=False, show_membrane=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    sql_node_filter=sql_filter
)

### Synapses

#### Load synapses

In [ ]:

for i in range(2):
    assembly_name = 'synapses_%d' % i
    sql_filter = 'presynaptic_neuron_guid=%d and guid%%2=%d' % (guid, i)
    be.remove_assembly(assembly_name)
    neurons_assembly = be.add_assembly(assembly_name)
    neurons_model = be.add_synapses(
        assembly_name=assembly_name,
        population_name=population_name,
        representation=be.SYNAPSE_REPRESENTATION_SPINE,
        realism_level=be.MORPHOLOGY_REALISM_LEVEL_SPINE,
        radius_multiplier=2.0,
        sql_filter=sql_filter
    )

In [ ]:
status = be.reset_camera()

### Materials and colors

In [ ]:
import seaborn as sns
def set_materials(
        model_id, color,
        shading_mode=be.shading_mode.NONE, user_param=1.0,
        emission=0.0, glossiness=1.0):
    material_ids = be.get_material_ids(model_id)['ids']
    shading_modes = list()
    colors = list()
    user_params = list()
    emissions = list()
    glossinesses = list()
    for _ in material_ids:
        colors.append(color)
        shading_modes.append(shading_mode)
        user_params.append(user_param)
        emissions.append(emission)
        glossinesses.append(glossiness)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        user_parameters=user_params, glossinesses=glossinesses,
        shading_modes=shading_modes, emissions=emissions
    )
    
model_ids = be.get_model_ids()['ids']
set_materials(model_ids[0], [0.5, 0.5, 0.5])
set_materials(model_ids[1], [0.0, 0.0, 1.0])
set_materials(model_ids[2], [0.9, 0.9, 0.2])
status = core.set_renderer()

### Rendering settings

In [ ]:
status = core.set_renderer(
    background_color=[1,1,1],
    current='advanced',subsampling=4, max_accum_frames=64)
params = core.AdvancedRendererParams()
params.gi_ray_length = 500.0
params.shadow_intensity = 1.0
params.soft_shadow_strength = 1.0
params.epsilon_multiplier = 10.0
params.max_ray_depth = 1
params.show_background = False
params.main_exposure = 1.0
status = core.set_renderer_params(params)
status = core.set_renderer()